# Data Visualization

## Libraries

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random
import json

# Taxon Class

In [24]:
class Taxon:
    def __init__(self, name):
        self.name = name
        self.parent = None
        self.children = []

    def set_parent(self, parent):
        self.parent = parent
        parent.add_child(self)

    def add_child(self, child):
        self.children.append(child)

class Species(Taxon):
    pass

class Genus(Taxon):
    pass

class Class(Taxon):
    pass

class Binary(Taxon):
    pass

class Object(Taxon):
    pass

class TaxonomyManager:
    def __init__(self):
        self.species = {}
        self.genera = {}
        self.classes = {}
        self.binaries = {}
        self.objects = {}

    def add_taxon(self, taxon, name, parent=None):
        if isinstance(taxon, Species):
            self.species[name] = taxon
        elif isinstance(taxon, Genus):
            self.genera[name] = taxon
        elif isinstance(taxon, Class):
            self.classes[name] = taxon
        elif isinstance(taxon, Binary):
            self.binaries[name] = taxon
        elif isinstance(taxon, Object):
            self.objects[name] = taxon
        if parent:
            taxon.set_parent(parent)

    def get_species(self, name):
        return self.species.get(name)

    def get_genus(self, name):
        return self.genera.get(name)

    def get_class(self, name):
        return self.classes.get(name)

    def get_binary(self, name):
        return self.binaries.get(name)

    def get_object(self, name):
        return self.objects.get(name)


# Populate Ontology

In [25]:
taxonomy = TaxonomyManager()

# Populate Objects
object_names = ["marine life", "inanimate"]
for name in object_names:
    taxonomy.add_taxon(Object(name), name)

# Populate Binaries
binary_mapping = {
    "asteroidea": "marine life",
    "phaeophyceae": "marine life",
    "bivalia": "marine life",
    "myxini": "marine life",
    "artificial": "inanimate",
    "natural": "inanimate",
    "chlorophyta": "marine life",
    "monocots": "marine life"
}
for class_name, binary_name in binary_mapping.items():
    binary = taxonomy.get_object(binary_name)
    taxonomy.add_taxon(Class(class_name), class_name, binary)

# Populate Classes
class_mapping = {
    "asterias" : "asteroidea",
    "fucus" : "phaeophyceae",
    "henrica" : "asteroidea",
    "mya": "bivalia",
    "myxine": "myxini",
    "cylindrical": "artificial",
    "solid": "natural",
    "arboral": "natural",
    "saccharina": "phaeophyceae",
    "ulva": "chlorophyta",
    "urospora": "chlorophyta",
    "zostera": "monocots"
}
for genus_name, class_name in class_mapping.items():
    class_ = taxonomy.get_class(class_name)
    taxonomy.add_taxon(Genus(genus_name), genus_name, class_)

# Populate Genera
genus_mapping = {
    "asterias rubens": "asterias",
    "asteroidea": "asterias",
    "fucus vesiculosus": "fucus",
    "henrica": "henrica",
    "mytilus edulis": "mya",
    "myxine glurinosa": "myxine",
    "pipe": "cylindrical",
    "rock": "solid",
    "saccharina latissima": "saccharina",
    "tree": "arboral",
    "ulva intestinalis": "ulva",
    "urospora": "urospora",
    "zostera marina": "zostera"
}
for species_name, genus_name in genus_mapping.items():
    genus = taxonomy.get_genus(genus_name)
    taxonomy.add_taxon(Species(species_name), species_name, genus)

In [23]:
# Get a species
asterias_rubens = taxonomy.get_species("ulva intestinalis")
print("Species:", asterias_rubens.name)

# Get the genus of a species
asterias_genus = asterias_rubens.parent
print("Genus:", asterias_genus.name)

# Get the class of a genus
asteroidea_class = asterias_genus.parent
print("Class:", asteroidea_class.name)

# Get the binary category of a class
marine_life_binary = asteroidea_class.parent
print("Binary Category:", marine_life_binary.name)

Species: ulva intestinalis
Genus: ulva
Class: chlorophyta
Binary Category: marine life


In [26]:
threshold = 0.75

In [28]:
# Image ID
target_image_id = "80_20220907 (4008).JPG" #"79_20220701 (284).JPG"

def classify_species_family(item, species_threshold=0.75):
    max_activation = max(item['activations'])
    max_species_index = item['activations'].index(max_activation)
    max_species = species_mapping[max_species_index]
    
    if max_activation < species_threshold:
        family = family_mapping[max_species]
        if family != "":
            max_species = family
    
    return max_species

# Buscar el item con el image_id objetivo
target_item = None
for item in predictions:
    if item['image_id'] == target_image_id:
        target_item = item
        break

if target_item is not None:
    final_classification = classify_species_family(target_item)
    
    original_classification = species_mapping[target_item['activations'].index(max(target_item['activations']))]
    
    if final_classification != original_classification:
        final_classification = f"{final_classification} ({original_classification})"
    
    print("Clasificación final:", final_classification)
else:
    print(f"No se encontró ningún item con image_id igual a '{target_image_id}'.")

No se encontró ningún item con image_id igual a '80_20220907 (4008).JPG'.


In [29]:
# Process each prediction and update the category_id and original_category_id
for prediction in predictions:

    class_found = False
    
    species_index = prediction['category_id']
    species_name = species_mapping.get(species_index)

    original_score = max(prediction['activations'])

    if species_name is not None:
        # Determine if the species should be updated based on activation threshold
        if original_score < threshold:
            family_name = family_mapping.get(species_name)

            if family_name:
                family_species_names = get_species_names_from_family_names(family_name)
                # Calculate upper category score
                family_score = 0
                for family_specie_name in family_species_names:
                    family_specie_id = get_specie_id_from_specie_name(family_specie_name)
                    family_score += prediction['activations'][family_specie_id]
                
                family_index = get_family_id_from_family_name(family_name)
                
                if family_score >= threshold:
                    class_found = True
                    # Update category_id and original_category_id
                    prediction['original_category_id'] = species_index
                    prediction['category_id'] = family_index
                    prediction['original_score'] = original_score
                    prediction['score'] = family_score
            
            if not class_found:
                # if there is an upper category
                prediction['original_category_id'] = species_index
                prediction['category_id'] = 1000 # animal idx
                prediction['original_score'] = original_score
                prediction['score'] = 1

In [30]:
# Save the modified predictions to a new JSON file
with open(new_predictions_file, 'w') as f:
    json.dump(predictions, f, indent=4)